In [1]:
# import json

# # prepare dataset in correct format
# dev_dataset = [json.loads(s) for s in open("/root/data/dev.json", "r").read().split('\n')[:-1]]
# input_list_of_dict = []
# for obj in dev_dataset:
#     instruction = obj['nl'].split("concode_field_sep")[0]
#     input = 'concode_field_sep'.join(obj['nl'].split("concode_field_sep")[1:])
#     input = input.replace('concode_elem_sep', '\n')
#     input = input.replace('concode_field_sep', '\n')
#     assert '\n' not in obj['code']
#     input_list_of_dict.append({"instruction": f"Generate java code\n{instruction}",
#                                "input": input,
#                                "output": obj['code']
#                               }
#                              )

# json.dump(input_list_of_dict, open("/root/data/final_dev.json", 'w+'))
# # json.dump('\n'.join([s['output'] for s in input_list_of_dict]), open("/root/data/final_dev.txt", 'w+'))
# open("/root/data/final_dev.txt", 'w+').write('\n'.join([s['output'] for s in input_list_of_dict]))

# open('/root/data/final_answers_jsonl.json', 'w+').\
# write(
#     '\n'.join([json.dumps({"code": obj['output']})
#                for obj in input_list_of_dict
#               ]
#              )
# )
#
# open("/root/data/final_answers_jsonl.json", "r").read().count('\n'),\
# open("/root/data/final_dev.json", "r").read().count('\n'),\
# open("/root/data/final_dev.txt", "r").read().count('\n')

In [5]:
exp_name = "t2c_concode_220428_v33"

In [6]:
!ls /root/data/

answers.json		  reference_corpus.txt	t2c_dev.json
dev.json		  t2c_10000train.json	t2c_test.json
final_answers_jsonl.json  t2c_1000train.json	t2c_train.json
final_dev.json		  t2c_10answers.json
final_dev.txt		  t2c_answers.json


In [13]:
import sys 
sys.path.append("/root/HSE_diploma/")

import pickle
from datetime import datetime


import numpy as np
import json
import pandas as pd
import os

EXPERIMENT_PATH = "/root/experiments/"
ARTIFACTS_PATH = "/root/temperary_results/"

df_experiments = pd.read_csv("all_experiments.csv").set_index("experiment_name_short")

line = df_experiments[df_experiments['exp_name'] == exp_name]
assert(len(line)==1)
line = line.iloc[0]
line = line.to_dict()

In [77]:
df_experiments[df_experiments['default_model']=='decapoda-research/llama-30b-hf']

,exp_name,adapter_model.bin,experiment_config.json,adapter_config.json,fn_config,experiment_name,fn_train_dataset,default_model,MICRO_BATCH_SIZE,BATCH_SIZE,...,bleu_batch_size,GRADIENT_ACCUMULATION_STEPS,fn_eval_dataset,log_bleu_steps_factor,eval_steps,evaluation_strategy,load_best_model_at_end,epoch,num_virtual_tokens,mapping_hidden_dim
experiment_name_short,,,,,,,,,,,,,,,,,,,,,
26,t2c_concode_220428_v26,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v26,/root/data/t2c_train.json,decapoda-research/llama-30b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,100.0,steps,NaN,NaN,NaN,NaN


In [78]:
df_experiments.loc[df_experiments['default_model'] == 'decapoda-research/llama-13b-hf', "exp_name"].unique()

array(['t2c_concode_220428_v21', 't2c_concode_220428_v24',
       't2c_concode_220428_v24_temp'], dtype=object)

In [ ]:
# !python3 evaluate_experiment.py "t2c_concode_220428_v24" experiment_not_on_main_node true

In [92]:
df_experiments.loc[df_experiments['default_model'] == 'decapoda-research/llama-13b-hf', "exp_name"].unique()

array(['t2c_concode_220428_v21', 't2c_concode_220428_v24',
       't2c_concode_220428_v24_temp'], dtype=object)

In [142]:
df_experiments.loc[df_experiments['default_model'] == 'decapoda-research/llama-7b-hf']

,exp_name,adapter_model.bin,experiment_config.json,adapter_config.json,fn_config,experiment_name,fn_train_dataset,default_model,MICRO_BATCH_SIZE,BATCH_SIZE,...,bleu_batch_size,GRADIENT_ACCUMULATION_STEPS,fn_eval_dataset,log_bleu_steps_factor,eval_steps,evaluation_strategy,load_best_model_at_end,epoch,num_virtual_tokens,mapping_hidden_dim
experiment_name_short,,,,,,,,,,,,,,,,,,,,,
14,t2c_concode_220428_v14,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v14,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,t2c_concode_220428_v15,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v15,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16_old,t2c_concode_220428_v16_old,False,True,False,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v16,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,NaN,NaN,NaN,NaN,NaN,NaN
16,t2c_concode_220428_v16,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v16,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,100.0,steps,NaN,NaN,NaN,NaN
18,t2c_concode_220428_v18,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v18,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,100.0,steps,NaN,NaN,NaN,NaN
19,t2c_concode_220428_v19,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v19,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,100.0,steps,NaN,NaN,NaN,NaN
20,t2c_concode_220428_v20,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v20,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,100.0,steps,NaN,NaN,NaN,NaN
22,t2c_concode_220428_v22,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v22,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,100.0,steps,NaN,NaN,NaN,NaN
30,t2c_concode_220428_v30,True,True,True,/root/experiments/experiments/t2c_concode_2204...,t2c_concode_220428_v30,/root/data/t2c_train.json,decapoda-research/llama-7b-hf,2.0,10.0,...,5.0,5.0,/root/data/t2c_answers.json,50.0,100.0,steps,True,NaN,NaN,NaN


In [147]:
a = set([s.split("t2c_concode_220428_v")[-1].split("_2023")[0] for s in os.listdir("/root/temperary_results/")])

In [153]:
df_experiments.loc[[i not in a for i in df_experiments.index]]['default_model'].apply(str).value_counts()

default_model
decapoda-research/llama-7b-hf     8
nan                               4
yahma/llama-7b-hf                 3
decapoda-research/llama-13b-hf    1
decapoda-research/llama-30b-hf    1
Name: count, dtype: int64

In [154]:
df_experiments.loc[[i in a for i in df_experiments.index]]['default_model'].apply(str).value_counts()

default_model
yahma/llama-7b-hf                 7
decapoda-research/llama-7b-hf     4
decapoda-research/llama-13b-hf    2
nan                               1
Name: count, dtype: int64

In [151]:
df_experiments.loc[[i in a for i in df_experiments.index]]['default_model'].value_counts()

default_model
yahma/llama-7b-hf                 7
decapoda-research/llama-7b-hf     4
decapoda-research/llama-13b-hf    2
Name: count, dtype: int64

In [143]:
!ls /root/temperary_results/

 predictionst2c_concode_220428_v10_2023-05-27-20:59:21.json
 predictionst2c_concode_220428_v10_2023-05-27-20:59:21.pickle
 predictionst2c_concode_220428_v14_2023-05-28-02:06:23.json
 predictionst2c_concode_220428_v14_2023-05-28-02:06:23.pickle
 predictionst2c_concode_220428_v15_2023-05-28-07:15:59.json
 predictionst2c_concode_220428_v15_2023-05-28-07:15:59.pickle
 predictionst2c_concode_220428_v16_2023-05-28-17:22:40.json
 predictionst2c_concode_220428_v16_2023-05-28-17:22:40.pickle
 predictionst2c_concode_220428_v16_old_2023-05-28-12:19:53.json
 predictionst2c_concode_220428_v16_old_2023-05-28-12:19:53.pickle
'predictionst2c_concode_220428_v21_2023-05-28-01 07 16.pickle'
'predictionst2c_concode_220428_v24_2023-05-28-18 57 46.pickle'
 predictionst2c_concode_220428_v33_2023-05-26-16:34:58.json
 predictionst2c_concode_220428_v33_2023-05-26-16:36:00.json
 predictionst2c_concode_220428_v33_2023-05-26-16:36:00.pickle
 predictionst2c_concode_220428_v33_2023-05-26-16:36:00.txt
 predictionst2c

In [50]:
!ls /root/temperary_results/

array(['decapoda-research/llama-7b-hf', 'decapoda-research/llama-13b-hf',
       'decapoda-research/llama-30b-hf', 'yahma/llama-7b-hf'],
      dtype=object)

In [5]:
# df_experiments

In [6]:
df_experiments.loc[["9", "10", "11", "12", "13"], "default_model"] = ["decapoda-research/llama-7b-hf"]*5

In [7]:
df_experiments[df_experiments['experiment_config.json']==0]

,exp_name,adapter_model.bin,experiment_config.json,adapter_config.json,fn_config,experiment_name,fn_train_dataset,default_model,MICRO_BATCH_SIZE,BATCH_SIZE,...,bleu_batch_size,GRADIENT_ACCUMULATION_STEPS,fn_eval_dataset,log_bleu_steps_factor,eval_steps,evaluation_strategy,load_best_model_at_end,epoch,num_virtual_tokens,mapping_hidden_dim
experiment_name_short,,,,,,,,,,,,,,,,,,,,,
9,t2c_concode_220428_v9,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,t2c_concode_220428_v10,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,t2c_concode_220428_v11,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,t2c_concode_220428_v12,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,t2c_concode_220428_v13,True,False,True,/root/experiments/experiments/t2c_concode_2204...,NaN,NaN,decapoda-research/llama-7b-hf,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# df_experiments[df_experiments[]]
# df_experiments['default_model']

In [9]:
# df_experiments
# os.system()

In [10]:
experiment_name = line["experiment_name"]

In [11]:
# Loading files to remote host if they are not exist
if not os.path.exists(os.path.join(EXPERIMENT_PATH, experiment_name)):
    print("loading files")
    #!mkdir {os.path.join(EXPERIMENT_PATH, experiment_name)}
    os.system(f"mkdir {os.path.join(EXPERIMENT_PATH, experiment_name)}")
    for filename in ['adapter_model.bin', 'experiment_config.json', 'adapter_config.json']:
        file_from = f"/root/experiments/experiments/{experiment_name}/{filename}"
        file_to   = f"/root/experiments/{experiment_name}/{filename}"
        print(file_from, '->', file_to)
        #!scp -i ~/.ssh/master_hetzner root@65.108.123.219:{file_from} {file_to} 
        os.system(f"scp -i ~/.ssh/master_hetzner root@65.108.123.219:{file_from} {file_to} ")
else:
    print("File already here")

File already here


In [12]:
!ls -lah /root/experiments/{exp_name}/

total 33M
drwxr-xr-x  2 root root  88 May 26 16:00 .
drwxr-xr-x 10 root root 279 May 26 20:52 ..
-rw-r--r--  1 root root 359 May 26 21:58 adapter_config.json
-rw-r--r--  1 root root 33M May 26 21:58 adapter_model.bin
-rw-r--r--  1 root root 815 May 26 21:58 experiment_config.json


In [13]:
# experiment_name = "/root/experiments/t2c_concode_220428_v38_plustwoepoch_test_copy/"
experiment_config = None
default_model = line['default_model']

if line['experiment_config.json']:
    experiment_config = json.load(open(f"{EXPERIMENT_PATH}/{experiment_name}/experiment_config.json", 
                                           "r"
                                          )
                                     )
    
    default_model = json.load(open(f"{EXPERIMENT_PATH}/{experiment_name}/experiment_config.json", 
                                   "r"
                                  )
                             )['default_model']

# params_iteration = {"temperature": [1.0],
#                     "max_new_tokens": [300]#None, 20, 30, 35, 40, 50, 60, 70, 80, 90, 100] + [45, 47, 49, 51, 53, 55]
#                    }

print("default_model=", default_model)
# print("params_iteration=", params_iteration)

default_model= yahma/llama-7b-hf


In [14]:
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer

from peft import PeftModel
import torch


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


In [15]:
model = LLaMAForCausalLM.from_pretrained(
        pretrained_model_name_or_path = default_model,
        load_in_8bit=True,
        device_map="auto"
    )

model = PeftModel.from_pretrained(
        model = model,
        model_id = os.path.join(EXPERIMENT_PATH, experiment_name),
        torch_dtype=torch.float16,
    )

tokenizer = LLaMATokenizer.from_pretrained(pretrained_model_name_or_path = default_model)

model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
tokenizer.padding_side = "left"
# not sure how necessary this part is, not sure if tloen/alpaca-lora-7b was even trained with EOS and BOS tokens
model.config.bos_token_id = 1
model.config.eos_token_id = 2

model.eval()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          

In [16]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    if hasattr(model, "print_trainable_parameters"):
        model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

# tokenizer.pad_token_id 0
# tokenizer.eos_token_id 2
# tokenizer.bos_token_id 1
# tokenizer.eos_token_id 2
# model.config.pad_token_id 0
# model.config.eos_token_id 2
# model.config.bos_token_id 1
# model.config.eos_token_id 2
# trainable params: 0 || all params: 6746804224 || trainable%: 0.0


tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.eos_token_id 2
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
model.config.eos_token_id 2
trainable params: 0 || all params: 6746804224 || trainable%: 0.0


In [14]:
from prompter import generate_test_prompt, get_response
from transformers import GenerationConfig
import math
import tqdm

/root/HSE_diploma/prompter/templates/


In [24]:
generation_config_dict = {"temperature": 1.0,
                          #"penalty_alpha": 2,
                          "max_new_tokens": 500
                         }

generation_config = GenerationConfig(**generation_config_dict)

fn_test_data = "/root/data/final_dev.json"
# fn_test_data = "../data/t2c_answers.json"
# fn_etalon = "/root/data/answers.json"
batch_size = 10
verbose = False

assert model.training == False

lst = json.load(open(fn_test_data, 'rb'))
inputs = lst

prompts = [generate_test_prompt(inp) for inp in inputs]
prompts = np.array(prompts)


In [25]:
lst[0]

{'instruction': 'Generate java code\nactually walks the bag to make sure the count is correct and resets the running total ',
 'input': ' Object _current \n int _total \n DefaultMapBag _parent \n Map _map \n int _mods \n Iterator _support \n boolean add \n boolean add \n Object next \n boolean containsAll \n boolean containsAll \n void clear \n boolean isEmpty \n boolean hasNext \n void remove \n boolean remove \n boolean remove \n Map getMap \n int modCount \n boolean contains \n Iterator iterator \n boolean removeAll \n int size \n boolean addAll \n int hashCode \n boolean equals \n Object[] toArray \n Object[] toArray \n Set uniqueSet \n void setMap \n String toString \n int getCount \n List extractList \n boolean retainAll \n boolean retainAll',
 'output': 'int function ( ) { _total = extractList ( ) . size ( ) ; return _total ; }'}

In [28]:
prompts[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate java code\nactually walks the bag to make sure the count is correct and resets the running total \n\n### Input:\n Object _current \n int _total \n DefaultMapBag _parent \n Map _map \n int _mods \n Iterator _support \n boolean add \n boolean add \n Object next \n boolean containsAll \n boolean containsAll \n void clear \n boolean isEmpty \n boolean hasNext \n void remove \n boolean remove \n boolean remove \n Map getMap \n int modCount \n boolean contains \n Iterator iterator \n boolean removeAll \n int size \n boolean addAll \n int hashCode \n boolean equals \n Object[] toArray \n Object[] toArray \n Set uniqueSet \n void setMap \n String toString \n int getCount \n List extractList \n boolean retainAll \n boolean retainAll\n\n### Response:\n'

In [29]:
res_list = []
n = math.ceil(len(prompts)/batch_size)

for ind in tqdm.tqdm(range(n)):
    current_prompts = prompts[ind*batch_size: (ind+1)*batch_size]
    if verbose:
        print(ind * batch_size, (ind+1)*batch_size, len(current_prompts))

    tokenized_inputs = tokenizer(list(current_prompts), 
                                 padding=True, 
                                 truncation=True, 
                                 return_tensors="pt"
                                ).to('cuda')



    with torch.no_grad():
        full_output = model.generate(
            **tokenized_inputs,
            generation_config=generation_config
        )

    res_list.extend(tokenizer.batch_decode(full_output, 
                                           skip_special_tokens=False
                                          )
                   )

100%|███████████████████████████████████████| 200/200 [1:12:17<00:00, 21.69s/it]


In [30]:
res_dict = {"real_config": experiment_config,
            "meta_config": line,
            "generation_config": generation_config_dict,
            "fn_test_data": fn_test_data,
#             "fn_etalon": fn_etalon,
            "batch_size": batch_size,
            "verbose": verbose,
            "prompts": list(prompts),
            "res_list": res_list
           }

fn_output = ARTIFACTS_PATH +\
            "predictions" +\
            exp_name+"_"+str(datetime.now()).split('.')[0].replace(' ', '-')
print(fn_output)

/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-27-00:11:08


In [31]:
json.dump(res_dict, 
          open(fn_output+'.json', "w+")
         )

pickle.dump(res_dict,
            open(fn_output+'.pickle', "wb")
           )

In [15]:
def preprocess(s):
    #ToDo rewrite it using Promt Template
    s = get_response(s)
    if "</s>" in s:
        s = s.split('</s>')[0]
    if "### Input" in s:
        s = s.split("### Input")[0]
    while '  ' in s:
        s = s.replace('  ', ' ')
    try:
        assert "<unk>" not in s
        assert "\n" not in s
        assert "  " not in s
    except Exception as e:
        print("Something wrong with")
        print(s)
        print('==')
        raise e


    return s
# preprocess("\n")

In [41]:
predictions_list = [preprocess(s) for s in res_list]

In [42]:
open(fn_output+".txt", 'w+').write('\n'.join(predictions_list))
# 9321

209372

In [43]:
fn_results = fn_output+".txt"

In [44]:
fn = fn_output+".txt"
print(fn)

/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-27-00:11:08.txt


In [46]:
print(ARTIFACTS_PATH)
!ls -lah {ARTIFACTS_PATH}

/root/temperary_results/
total 19M
drwxr-xr-x 2 root root 4.0K May 27 00:18 .
drwx------ 1 root root 4.0K May 26 16:35 ..
-rw-r--r-- 1 root root 2.0K May 26 16:47 predictionst2c_concode_220428_v33_2023-05-26-16:34:58.json
-rw-r--r-- 1 root root 261K May 26 16:36 predictionst2c_concode_220428_v33_2023-05-26-16:36:00.json
-rw-r--r-- 1 root root 474K May 26 16:36 predictionst2c_concode_220428_v33_2023-05-26-16:36:00.pickle
-rw-r--r-- 1 root root 9.2K May 26 16:41 predictionst2c_concode_220428_v33_2023-05-26-16:36:00.txt
-rw-r--r-- 1 root root 261K May 26 16:46 predictionst2c_concode_220428_v33_2023-05-26-16:46:49.json
-rw-r--r-- 1 root root 474K May 26 16:46 predictionst2c_concode_220428_v33_2023-05-26-16:46:49.pickle
-rw-r--r-- 1 root root 9.2K May 26 16:46 predictionst2c_concode_220428_v33_2023-05-26-16:46:49.txt
-rw-r--r-- 1 root root 261K May 26 20:45 predictionst2c_concode_220428_v33_2023-05-26-20:45:26.json
-rw-r--r-- 1 root root 474K May 26 20:45 predictionst2c_concode_220428_v33_2

In [155]:
0

0

In [23]:
import re
# "/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-26-16:46:49.txt"
def run_evaluation_of_metrics(fn):
    #fn_refference = "/root/data/reference_corpus.txt"
    #fn_answers = "/root/data/answers.json"
    
    fn_refference = "/root/data/final_dev.txt"
    fn_answers = "/root/data/final_answers_jsonl.json"
    open("/root/HSE_diploma/load_all_experiments/res.txt", "w+").write('')
    print(f"/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs {fn_refference} --hyp $fn --lang java --params 0.25,0.25,0.25,0.25")
    !cd /root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs {fn_refference} --hyp $fn --lang java --params 0.25,0.25,0.25,0.25 > /root/HSE_diploma/load_all_experiments/res.txt
    !cd /root/HSE_diploma/evaluateFromCodeXGlue/;python3 calculate_bleu/evaluator.py -a={fn_answers} -p=$fn  >> /root/HSE_diploma/load_all_experiments/res.txt


def parse_metrics(fn_metrics = "res.txt"):
    s = open('res.txt', "r").read()

    BLEU = re.findall("BLEU: \d+\.\d+", s)
    EM = re.findall("EM: \d+\.\d+", s)
    CodeBLEU = re.findall("CodeBLEU score:  \d+\.\d+", s)

    assert len(BLEU) == len(EM) == len(CodeBLEU) == 1
    BLEU = float(BLEU[0].split(' ')[-1])
    EM = float(EM[0].split(' ')[-1])
    CodeBLEU = float(CodeBLEU[0].split(' ')[-1])

    ngram_match = float(re.findall("ngram match: \d+\.\d+", s)[0].split(' ')[-1]) 
    weighted_ngram_match = float(re.findall("weighted ngram match: \d+\.\d+", s)[0].split(' ')[-1]) 
    syntax_match = float(re.findall("syntax_match: \d+\.\d+", s)[0].split(' ')[-1]) 
    dataflow_match = float(re.findall("dataflow_match: \d+\.\d+", s)[0].split(' ')[-1]) 

    precisions = re.findall("precisions:\s+.+\n", s)[0].split(':  ')[-1][:-1]
    bp = float(re.findall("bp:  \d+\.\d+", s)[0].split(' ')[-1])
    ratio = float(re.findall("ratio:  \d+\.\d+", s)[0].split(' ')[-1])
    translation_length = int(re.findall("translation_length:.*\n", s)[0].split(' ')[-1][:-1])
    reference_length = int(re.findall("reference_length:.*\n", s)[0].split(' ')[-1][:-1])

    # weighted ngram match: 0.2792407600329364, 
    # syntax_match: 0.3445378151260504, 
    # dataflow_match: 0.2908777969018933
    metrics = {"BLEU": BLEU,
               "EM": EM,
               "CodeBLEU": CodeBLEU,
               "ngram_match": ngram_match,
               "weighted_ngram_match": weighted_ngram_match,
               "syntax_match": syntax_match,
               "dataflow_match": dataflow_match,
               "precisions": precisions,
               "bp": bp,
               "ratio": ratio,
               "translation_length": translation_length,
               "reference_length": reference_length,
               "raw": s
              }
    return metrics


In [47]:

run_evaluation_of_metrics(fn = fn_results)
final_metrics = parse_metrics()

final_metrics

/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 27.33, EM: 15.55


{'BLEU': 27.33,
 'EM': 15.55,
 'CodeBLEU': 0.3021318649354401,
 'ngram_match': 0.2730237442281213,
 'weighted_ngram_match': 0.2827310227335677,
 'syntax_match': 0.33810699116060156,
 'dataflow_match': 0.31466570161946983,
 'precisions': '[0.632047216349542, 0.4579639394846524, 0.3269977568199908, 0.23599260703344557]',
 'bp': 0.706824459129736,
 'ratio': 0.7424054151270396,
 'translation_length': 45407,
 'reference_length': 61162,
 'raw': 'ngram match: 0.2730237442281213, weighted ngram match: 0.2827310227335677, syntax_match: 0.33810699116060156, dataflow_match: 0.31466570161946983\nCodeBLEU score:  0.3021318649354401\nBLEU: 27.33, EM: 15.55\nprecisions:  [0.632047216349542, 0.4579639394846524, 0.3269977568199908, 0.23599260703344557]\nbp:  0.706824459129736\nratio:  0.7424054151270396\ntranslation_length:  45407\nreference_length:  61162\n'}

In [ ]:

run_evaluation_of_metrics(fn = fn_results)
final_metrics = parse_metrics()

final_metrics

/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 18.27, EM: 10.0


{'BLEU': 18.27,
 'EM': 10.0,
 'CodeBLEU': 0.2310302252826384,
 'ngram_match': 0.1807804463258241,
 'weighted_ngram_match': 0.23934755329629998,
 'syntax_match': 0.2795031055900621,
 'dataflow_match': 0.22448979591836735,
 'precisions': '[0.875968992248062, 0.6470588235294118, 0.44036697247706424, 0.30303030303030304]',
 'bp': 0.34830125444308524,
 'ratio': 0.4866920152091255,
 'translation_length': 128,
 'reference_length': 263,
 'raw': 'ngram match: 0.1807804463258241, weighted ngram match: 0.23934755329629998, syntax_match: 0.2795031055900621, dataflow_match: 0.22448979591836735\nCodeBLEU score:  0.2310302252826384\nBLEU: 18.27, EM: 10.0\nprecisions:  [0.875968992248062, 0.6470588235294118, 0.44036697247706424, 0.30303030303030304]\nbp:  0.34830125444308524\nratio:  0.4866920152091255\ntranslation_length:  128\nreference_length:  263\n'}

In [153]:
res_dict['final_metrics'] = final_metrics

In [154]:
json.dump(res_dict, 
          open(fn_output+'_final_metrics.json', "w+")
         )

pickle.dump(res_dict,
            open(fn_output+'_final_metrics.pickle', "wb")
           )

In [155]:
fn_output

'/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-26-22:34:18'

['predictionst2c_concode_220428_v33_2023-05-26-16:36:00.pickle',
 'predictionst2c_concode_220428_v33_2023-05-26-16:46:49.pickle',
 'predictionst2c_concode_220428_v33_2023-05-26-20:45:26.pickle',
 'predictionst2c_concode_220428_v33_2023-05-26-20:45:26_final_metrics.pickle',
 'predictionst2c_concode_220428_v33_2023-05-26-22:34:18.pickle',
 'predictionst2c_concode_220428_v33_2023-05-26-22:34:18_final_metrics.pickle',
 'predictionst2c_concode_220428_v33_2023-05-27-00:11:08.pickle',
 'predictionst2c_concode_220428_v33_2023-05-27-02:01:01.pickle',
 'predictionst2c_concode_220428_v33_old_2023-05-27-03:10:51.pickle',
 'predictionst2c_concode_220428_v37_2023-05-27-04:41:10.pickle',
 'predictionst2c_concode_220428_v38_plusoneepoch_2023-05-27-07:44:33.pickle',
 'predictionst2c_concode_220428_v41_ptuning_2023-05-27-09:46:08.pickle',
 'predictionst2c_concode_220428_v42_ptuning_2023-05-27-13:18:30.pickle']

In [ ]:
# [i for i in os.listdir("/root/temperary_results/") if i.endswith('.pickle')]
all_res = []
for filename in [i for i in os.listdir("/root/temperary_results/") if i.endswith('.pickle')]:
    print(filename)
    fn_aggregation_metrics = f"/root/temperary_results/{filename}"
    dct = pickle.load(open(fn_aggregation_metrics, "rb"))
    if 'experiment_name' in dct['meta_config']:
        print(dct['meta_config']['experiment_name'])

    def preprocess(s):
        #ToDo rewrite it using Promt Template
        s = get_response(s)
        if "</s>" in s:
            s = s.split('</s>')[0]
        if "### Input" in s:
            s = s.split("### Input")[0]
        if "###" in s:
            s = s.split("###")[0]
        while '\n' in s:
            s = s.replace('\n', ' ')
        while '<unk>' in s:
            s = s.replace('<unk>', ' ')
        while '  ' in s:
            s = s.replace('  ', ' ')
        try:
            assert "<unk>" not in s
            assert "\n" not in s
            assert "  " not in s
        except Exception as e:
            print("Something wrong with")
            print(s)
            print('==')
            raise e


        return s

    predictions_list = [preprocess(s) for s in dct['res_list']]
    open("temp_230527.txt", 'w+').write('\n'.join(predictions_list))
    try:
        run_evaluation_of_metrics(fn = "/root/HSE_diploma/load_all_experiments/temp_230527.txt")
        final_metrics = parse_metrics()
    except:
        continue
    dct['final_metrics'] = final_metrics
    dct['fn_aggregation_metrics'] = fn_aggregation_metrics
    exp_name_from_file = filename.split("_2023")[0].split("concode_220428_")[-1]
    dct['exp_name_from_file'] = exp_name_from_file
    
    for i in [0, 1, 2, 3, 4, 5, 6, 7, 8, 666, 1000, -1]:
        dct[f'prediction[%d]'%i] = predictions_list[i]
    
    all_res.append(dct)

predictionst2c_concode_220428_v33_2023-05-26-16:36:00.pickle
t2c_concode_220428_v33
/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
Traceback (most recent call last):
  File "/root/HSE_diploma/evaluateFromCodeXGlue/calc_code_bleu.py", line 33, in <module>
    assert len(hypothesis) == len(pre_references[i])
AssertionError
Traceback (most recent call last):
  File "/root/HSE_diploma/evaluateFromCodeXGlue/calculate_bleu/evaluator.py", line 49, in <module>
    main()
  File "/root/HSE_diploma/evaluateFromCodeXGlue/calculate_bleu/evaluator.py", line 21, in main
    assert len(preds) == len(gts), f"Samples of predictions and answers are not equal, {len(preds)}: {len(gts)}"
AssertionError: Samples of predictions and answers are not equal, 100: 2000
predictionst2c_concode_220428_v33_2023-05-26-16:46:49.pickle
t2c_concode_220428_v33
/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bl

t2c_concode_220428_v15
/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 10.29, EM: 12.3
predictionst2c_concode_220428_v16_old_2023-05-28-12:19:53.pickle
t2c_concode_220428_v16
/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 16.13, EM: 12.75
predictionst2c_concode_220428_v16_2023-05-28-17:22:40.pickle
t2c_concode_220428_v16
/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 16.13, EM: 12.75
predictionst2c_concode_220428_v24_2023-05-28-18 57 46.pickle
/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 15.61, EM: 8.6

In [174]:
# len(all_res[-1])
# all_res[-1].keys()
!pwd

/root/HSE_diploma/load_all_experiments


In [175]:
pickle.dump(all_res, open("all_res_v1.pickle", "wb"))

In [176]:
!ls -lah

total 229M
drwxr-xr-x  4 root root  275 Jun  2 16:09 .
drwxr-xr-x 15 root root 4.0K May 26 15:37 ..
drwxr-xr-x  2 root root  106 May 26 22:05 .ipynb_checkpoints
drwxr-xr-x  2 root root    6 May 26 16:28 ARTIFACTS_PATH
-rw-r--r--  1 root root  11K May 26 15:37 all_experiments.csv
-rw-r--r--  1 root root 227M Jun  2 16:09 all_res_v1.pickle
-rwxr-xr-x  1 root root 1.4K May 27 00:49 evaluate_all.sh
-rw-r--r--  1 root root  11K May 27 00:44 evaluate_experiment.py
-rw-r--r--  1 root root  43K May 26 22:07 load_all_from_main_node-old.ipynb
-rw-r--r--  1 root root 232K Jun  2 16:09 load_all_from_main_node.ipynb
-rw-r--r--  1 root root  388 Jun  2 16:01 res.txt
-rw-r--r--  1 root root 848K Jun  2 16:01 temp_230527.txt


In [121]:
# !ls t2c_concode_220428_v33
# l
# {i['meta_config']['experiment_name'] if 'experiment_name' in i['meta_config'] else i['fn_aggregation_metrics']: {k:v for k, v in i['final_metrics'].items() 
len([i for i in all_res if not isinstance(i['fn_aggregation_metrics'], str)])

0

In [172]:
for i in all_res:
    print(i['prediction[0]'])

void function ( ) { if ( _map
void function ( ) { if ( _map
void function ( ) { if ( _map
void function ( ) { _total = 0 ; for ( Iterator loc0 = iterator ( ) ; loc0 . hasNext ( ) ; ) { Object loc1 = loc0 . next ( ) ; _total += loc1 . hashCode ( ) ; } }
void function ( ) { if ( _parent != null ) { _parent . walk ( ) ; } _total = 0 ; }
void function ( ) { _total = _map . size ( ) ; }
void function ( ) { _map . clear ( ) ; _total = 0 ; _mods = 0 ; _current = null ; _support = _map . iterator ( ) ; }
void function ( ) { _total = 0 ; _support = _map . entrySet ( ) . iterator ( ) ; }
void function ( ) { _total = 0 ; _mods = 0 ; _current = null ; _map = null ; }
void function ( ) { _total = 0 ; } ### Exception : void function ( ) { throw new UnsupportedOperationException ( "srini_string" ) ; } ### Exception : void function ( ) { throw new UnsupportedOperationException ( "srini_string" ) ; } ### Exception : void function ( ) { throw new UnsupportedOperationException ( "srini_string" ) ; } ### 

In [169]:
pd.DataFrame({i['fn_aggregation_metrics']: i                             
  for i in all_res})

,/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-27-00:11:08.pickle,/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-27-02:01:01.pickle,/root/temperary_results/predictionst2c_concode_220428_v33_old_2023-05-27-03:10:51.pickle,/root/temperary_results/predictionst2c_concode_220428_v37_2023-05-27-04:41:10.pickle,/root/temperary_results/predictionst2c_concode_220428_v38_plusoneepoch_2023-05-27-07:44:33.pickle,/root/temperary_results/predictionst2c_concode_220428_v41_ptuning_2023-05-27-09:46:08.pickle,/root/temperary_results/predictionst2c_concode_220428_v42_ptuning_2023-05-27-13:18:30.pickle,/root/temperary_results/predictionst2c_concode_220428_v53_ptuning_2023-05-27-03 03 07.pickle,/root/temperary_results/predictionst2c_concode_220428_v43_ptuning_2023-05-27-15:54:37.pickle,/root/temperary_results/predictionst2c_concode_220428_v10_2023-05-27-20:59:21.pickle,...,/root/temperary_results/predictionst2c_concode_220428_v15_2023-05-28-07:15:59.pickle,/root/temperary_results/predictionst2c_concode_220428_v16_old_2023-05-28-12:19:53.pickle,/root/temperary_results/predictionst2c_concode_220428_v16_2023-05-28-17:22:40.pickle,/root/temperary_results/predictionst2c_concode_220428_v24_2023-05-28-18 57 46.pickle,/root/temperary_results/predictionst2c_concode_220428_v21_2023-05-28-01 07 16.pickle,/root/temperary_results/predictionst2c_concode_220428_v18_2023-05-28-22:24:35.pickle,/root/temperary_results/predictionst2c_concode_220428_v19_2023-05-29-03:32:37.pickle,/root/temperary_results/predictionst2c_concode_220428_v20_2023-05-29-08:39:36.pickle,/root/temperary_results/predictionst2c_concode_220428_v22_2023-05-29-13:44:32.pickle,/root/temperary_results/predictionst2c_concode_220428_v30_2023-05-29-18:53:00.pickle
real_config,"{'experiment_name': 't2c_concode_220428_v33', ...","{'experiment_name': 't2c_concode_220428_v33', ...","{'experiment_name': 't2c_concode_220428_v33', ...","{'experiment_name': 't2c_concode_220428_v37', ...",{'experiment_name': 't2c_concode_220428_v38_pl...,{'experiment_name': 't2c_concode_220428_v41_pt...,{'experiment_name': 't2c_concode_220428_v42_pt...,None,{'experiment_name': 't2c_concode_220428_v43_pt...,None,...,"{'experiment_name': 't2c_concode_220428_v15', ...","{'experiment_name': 't2c_concode_220428_v16', ...","{'experiment_name': 't2c_concode_220428_v16', ...",None,None,"{'experiment_name': 't2c_concode_220428_v18', ...","{'experiment_name': 't2c_concode_220428_v19', ...","{'experiment_name': 't2c_concode_220428_v20', ...","{'experiment_name': 't2c_concode_220428_v22', ...","{'experiment_name': 't2c_concode_220428_v30', ..."
meta_config,"{'exp_name': 't2c_concode_220428_v33', 'adapte...","{'exp_name': 't2c_concode_220428_v33', 'adapte...","{'exp_name': 't2c_concode_220428_v33_old', 'ad...","{'exp_name': 't2c_concode_220428_v37', 'adapte...",{'exp_name': 't2c_concode_220428_v38_plusoneep...,"{'exp_name': 't2c_concode_220428_v41_ptuning',...","{'exp_name': 't2c_concode_220428_v42_ptuning',...",{'experiment_config.json': False},"{'exp_name': 't2c_concode_220428_v43_ptuning',...","{'exp_name': 't2c_concode_220428_v10', 'adapte...",...,"{'exp_name': 't2c_concode_220428_v15', 'adapte...","{'exp_name': 't2c_concode_220428_v16_old', 'ad...","{'exp_name': 't2c_concode_220428_v16', 'adapte...",{'experiment_config.json': False},{'experiment_config.json': False},"{'exp_name': 't2c_concode_220428_v18', 'adapte...","{'exp_name': 't2c_concode_220428_v19', 'adapte...","{'exp_name': 't2c_concode_220428_v20', 'adapte...","{'exp_name': 't2c_concode_220428_v22', 'adapte...","{'exp_name': 't2c_concode_220428_v30', 'adapte..."
generation_config,"{'temperature': 1.0, 'max_new_tokens': 500}","{'temperature': 1.0, 'max_new_tokens': 500}","{'temperature': 1.0, 'max_new_tokens': 500}","{'temperature': 1.0, 'max_new_tokens': 500}","{'temperature': 1.0, 'max_new_tokens': 500}","{'temperature': 1.0, 'max_new_tokens': 500}","{'temperature': 1.0, 'max_new_tokens': 500}","{'temperature': 1.0, 'max_new_tokens': 50

In [122]:
pd.DataFrame({i['fn_aggregation_metrics']: {k:v for k, v in i['final_metrics'].items() 
                                        if k in ["BLEU", "EM", "CodeBLEU"]
                                       }
  for i in all_res}).T[["CodeBLEU", "BLEU", "EM"]]

,CodeBLEU,BLEU,EM
/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-27-00:11:08.pickle,0.302313,27.33,15.55
/root/temperary_results/predictionst2c_concode_220428_v33_2023-05-27-02:01:01.pickle,0.302358,27.33,15.55
/root/temperary_results/predictionst2c_concode_220428_v33_old_2023-05-27-03:10:51.pickle,0.302290,27.33,15.55
/root/temperary_results/predictionst2c_concode_220428_v37_2023-05-27-04:41:10.pickle,0.339938,31.78,18.10
/root/temperary_results/predictionst2c_concode_220428_v38_plusoneepoch_2023-05-27-07:44:33.pickle,0.352286,32.69,17.10
/root/temperary_results/predictionst2c_concode_220428_v41_ptuning_2023-05-27-09:46:08.pickle,0.317192,29.43,16.30
/root/temperary_results/predictionst2c_concode_220428_v42_ptuning_2023-05-27-13:18:30.pickle,0.283428,18.09,14.30
/root/temperary_results/predictionst2c_concode_220428_v53_ptuning_2023-05-27-03 03 07.pickle,0.326427,30.00,17.35
/root/temperary_results/predictionst2c_concode_220428_v43_ptuning_2023-05-27-15:54:37.pickle,0.303630,26.74,15.90
/root/temperary_results/predictionst2c_concode_220428_v10_2023-05-27-20:59:21.pickle,0.365946,3.36,0.00


In [116]:
all_res[:8][-1]['fn_aggregation_metrics'].split('?')

'/root/temperary_results/predictionst2c_concode_220428_v53_ptuning_2023-05-27-03 03 07.pickle'

In [ ]:
all_res[-1]['meta_config']['experiment_name'] 

In [98]:
all_res[-1]['meta_config']['experiment_name'] = "t2c_concode_220428_v53_ptuning"

In [74]:
# all_res[7]['real_config']

In [34]:
dct['final_metrics']

KeyError: 'final_metrics'

In [128]:
# dct['meta_config']
# dct
dct = pd.read_pickle("/root/temperary_results/predictionst2c_concode_220428_v37_2023-05-27-04:41:10.pickle")
#predictionst2c_concode_220428_v38_plusoneepoch_2023-05-27-07\ 44\ 33.json
# json.load(open("predictionst2c_concode_220428_v38_plusoneepoch_2023-05-27-07 44 33.json"))

In [129]:
dct.keys()

dict_keys(['real_config', 'meta_config', 'generation_config', 'fn_test_data', 'batch_size', 'verbose', 'prompts', 'res_list'])

In [134]:
# print(dct['prompts'][0])
print(dct['res_list'][43])

<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate java code
create an instance of providedportid 

### Input:
 QName _PropertyDeclaration_QNAME 
 QName _ProvidedPort_QNAME 
 QName _ApplicationPayload_QNAME 
 QName _Application_QNAME 
 QName _ApplicationID_QNAME 
 QName _ApplicationReleaseBinding_QNAME 
 QName _ConsumedPort_QNAME 
 QName _ProvidedPortID_QNAME 
 ConsumedPort createConsumedPort 
 JAXBElement<ConsumedPort> createConsumedPort 
 ApplicationPayload createApplicationPayload 
 JAXBElement<ApplicationPayload> createApplicationPayload 
 ApplicationReleaseBinding createApplicationReleaseBinding 
 JAXBElement<ApplicationReleaseBinding> createApplica

In [20]:
def preprocess(s):
    #ToDo rewrite it using Promt Template
    s = get_response(s)
    if "</s>" in s:
        s = s.split('</s>')[0]
    if "### Input" in s:
        s = s.split("### Input")[0]
    while '  ' in s:
        s = s.replace('  ', ' ')
    try:
        assert "<unk>" not in s
        assert "\n" not in s
        assert "  " not in s
    except Exception as e:
        print("Something wrong with")
        print(s)
        print('==')
        raise e


    return s

predictions_list = [preprocess(s) for s in dct['res_list']]

In [21]:
open("temp_230527.txt", 'w+').write('\n'.join(predictions_list))

232891

In [27]:
# !pwd
# !nvidia-smi

In [28]:
run_evaluation_of_metrics(fn = "/root/HSE_diploma/load_all_experiments/temp_230527.txt")
final_metrics = parse_metrics()

final_metrics

/root/HSE_diploma/evaluateFromCodeXGlue/;python3 calc_code_bleu.py --refs /root/data/final_dev.txt --hyp $fn --lang java --params 0.25,0.25,0.25,0.25
INFO:__main__:BLEU: 31.78, EM: 18.1


{'BLEU': 31.78,
 'EM': 18.1,
 'CodeBLEU': 0.33959841829122134,
 'ngram_match': 0.3178090896255435,
 'weighted_ngram_match': 0.3202477733364106,
 'syntax_match': 0.37409597061187005,
 'dataflow_match': 0.3462408395910613,
 'precisions': '[0.5986652683763943, 0.44099514322529487, 0.3195789039116524, 0.23513833566584133]',
 'bp': 0.8468476593360381,
 'ratio': 0.8574605146986691,
 'translation_length': 52444,
 'reference_length': 61162,
 'raw': 'ngram match: 0.3178090896255435, weighted ngram match: 0.3202477733364106, syntax_match: 0.37409597061187005, dataflow_match: 0.3462408395910613\nCodeBLEU score:  0.33959841829122134\nBLEU: 31.78, EM: 18.1\nprecisions:  [0.5986652683763943, 0.44099514322529487, 0.3195789039116524, 0.23513833566584133]\nbp:  0.8468476593360381\nratio:  0.8574605146986691\ntranslation_length:  52444\nreference_length:  61162\n'}